# Process data city company data

In [1]:
#import required libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [14]:
# read the spreadsheet containing all the data
scot_mfg_df = pd.read_csv('company data/basic_companies_list.csv')
SIC_des_df = pd.read_csv('company data/SIC_condensed_list.csv')

# User defined columns
col = ['Companyname', 'BestEstimateEmployees', 'BestEstimateTurnover','Postcodes','SICs']

# Seperate the defined columns and rename
scot_mfg_df = scot_mfg_df[col]
scot_mfg_df.rename(columns = {"Companyname":"Company name","BestEstimateEmployees":"Best estimate employees",
                   "BestEstimateTurnover":"Best estimate turnover","Postcodes":"Postcode",
                   "SICs":"SIC Code"}, inplace = True)

# Replace items with muliple SICs with the first recorded SIC and add respective SIC description 
# also for items with mulitple postcodes select only scotland postcodes
SIC_des_df.set_index('SIC Code',drop=True, append=False, inplace=True)
description =[]
idx=[]

for i in range (len(scot_mfg_df)):
    SIC_code = str(scot_mfg_df['SIC Code'].values[i]).split(',')[0]
    scot_mfg_df['SIC Code'].values[i] = SIC_code
    description.append(SIC_des_df['Description'].loc[int(SIC_code)])
    
    postcodes = str(scot_mfg_df['Postcode'].values[i]).split(',')
    selected_postcode = [item for item in postcodes if item.startswith('AB')or item.startswith('DD') or 
                         item.startswith('EH') or (item.startswith('G') and item[1].isdigit())]
    if len(selected_postcode)==0: 
        idx.append(i)
    else:
        scot_mfg_df['Postcode'].values[i] = selected_postcode[0]

scot_mfg_df['Description'] = description
scot_mfg_df.drop(scot_mfg_df.index[idx], axis=0, inplace=True)

# Reset index 
scot_mfg_df.reset_index(drop =True, inplace=True)
scot_mfg_df

,Company name,Best estimate employees,Best estimate turnover,Postcode,SIC Code,Description
0,ZYNQ LTD,0,28554,AB25 1TW,26309,Manufacture of communication equipment other t...
1,ZONAL RETAIL DATA SYSTEMS LIMITED,587,89124000,EH22 1SP,26200,Manufacture of computers and peripheral equipment
2,XEROX LIMITED,556,812000000,AB24 5AS,28230,Manufacture of office machinery and equipment ...
3,WTG TREATMENT LIMITED,4,1645000,EH54 7AW,16100,Sawmilling and planing of wood
4,YES PRINTING LIMITED,3,140000,G41 4AN,18129,Printing n.e.c.
...,...,...,...,...,...,...
548,A CAMERON (BOOKBINDERS) LTD,3,40000,G5 8DL,18140,Binding and related services
549,3D STUFF LIMITED,1,65000,EH5 1PP,25620,Machining
550,1STOPWIND LTD,13,910000,G51 3TR,33190,Repair of other equipment
551,-EASY FIX- REPAIR CENTER LTD,0,0,G5 9EQ,33140,Repair of electrical equipment


In [15]:
#Intialise geolocator and arrays to find latitiude and longitide for given postcodes
geolocator = Nominatim(timeout=10, user_agent="PDS")
location_latitude = []
location_longitude = [] 
idx=[]

for i in range(len(scot_mfg_df)):
    try:
        location = geolocator.geocode(scot_mfg_df["Postcode"][i])
        location_latitude.append(location.latitude)
        location_longitude.append(geolocator.geocode(scot_mfg_df["Postcode"][i]).longitude)
    except:
        print(scot_mfg_df["Postcode"][i]+',location not found')
        idx.append(i)
        
# Remove rows with undefined locations        
scot_mfg_df.drop(scot_mfg_df.index[idx], axis=0, inplace=True)
scot_mfg_df['latitude'] = location_latitude
scot_mfg_df['longitude'] = location_longitude

# Set index according to SIC description
scot_mfg_df.set_index('Description', drop=True, append=False, inplace=True)
scot_mfg_df

EH12 7BF,location not found
G46 8JZ,location not found
AB53 4PT,location not found
AB21 0ES,location not found
DD4 7BT,location not found
G64 2HX,location not found


,Company name,Best estimate employees,Best estimate turnover,Postcode,SIC Code,latitude,longitude
Description,,,,,,,
"Manufacture of communication equipment other than telegraph, and telephone apparatus and equipment",ZYNQ LTD,0,28554,AB25 1TW,26309,57.148960,-2.113850
Manufacture of computers and peripheral equipment,ZONAL RETAIL DATA SYSTEMS LIMITED,587,89124000,EH22 1SP,26200,55.913810,-3.090430
Manufacture of office machinery and equipment (except computers and peripheral equipment),XEROX LIMITED,556,812000000,AB24 5AS,28230,57.150458,-2.095051
Sawmilling and planing of wood,WTG TREATMENT LIMITED,4,1645000,EH54 7AW,16100,55.880504,-3.530149
Printing n.e.c.,YES PRINTING LIMITED,3,140000,G41 4AN,18129,55.830800,-4.294400
...,...,...,...,...,...,...,...
Binding and related services,A CAMERON (BOOKBINDERS) LTD,3,40000,G5 8DL,18140,55.851560,-4.260750
Machining,3D STUFF LIMITED,1,65000,EH5 1PP,25620,55.983230,-3.232854
Repair of other equipment,1STOPWIND LTD,13,910000,G51 3TR,33190,55.857330,-4.322460


In [16]:
scot_mfg_df.to_pickle('company data/scot_mfg_df.pkl')